In [ ]:
import tracemalloc
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import clear_output
clear_output()
tf.__version__

# Preparing the inputs of the model

In [ ]:
df = pd.read_csv('../input/heart-disease-uci/heart.csv')
df.head()

In [ ]:
X = df.drop('target',axis = 1)
y = df['target']

In [ ]:
feature = []
for i in range(len(X.columns)):
    feature.append(str(i))

col_rename = {i: j for i, j in zip(X.columns, feature)}
X = X.rename(columns=col_rename, inplace=False)
X = X.astype("int64")
y = y.astype("int64")

In [ ]:
def _make_input_fn(X, y, n_epochs=None, shuffle=True):
    def input_fn():
        NUM_EXAMPLES = len(y)
        dataset = tf.data.Dataset.from_tensor_slices((dict(X), y))
        if shuffle:
            dataset = dataset.shuffle(NUM_EXAMPLES)
        # For training, cycle thru dataset as many times as need (n_epochs=None).
        dataset = dataset.repeat(n_epochs)
        # In memory training doesn't use batching.
        dataset = dataset.batch(NUM_EXAMPLES)
        return dataset
    return input_fn

In [ ]:
train_input_fn = _make_input_fn(X, y)

num_columns = feature
feature_columns = []
n_classes = len(np.unique(y))

for feature_name in num_columns:
    feature_columns.append(tf.feature_column.numeric_column(feature_name,
                                                            dtype=tf.float32))

# Training the model

In [ ]:
T = 100
step = 1
t = range(1, T + 1, step)
mem = np.zeros((T,))
for i, tree in enumerate(t):
    tracemalloc.start()
    tracemalloc.clear_traces()
    zero_size = tracemalloc.get_traced_memory()[0]

    est = tf.estimator.BoostedTreesClassifier(feature_columns,
                                              n_batches_per_layer=1,
                                              n_classes=n_classes,
                                              n_trees=tree,
                                              max_depth=10,
                                              learning_rate=0.1,
                                              label_vocabulary=None,
                                              center_bias=False
                                              )
    est.train(train_input_fn, max_steps=None,
              steps=100)
    mem[i] = tracemalloc.get_traced_memory()[0] - zero_size
    clear_output()

# Memory usage

In [ ]:
fig, ax1 = plt.subplots()
ax1.set_xlabel("Booosting iteration")
ax1.set_ylabel("Memory usage (in MB)")
ln1 = ax1.plot(mem[1:], color='black',
               linewidth=1, drawstyle="steps-mid")
plt.title("Recorded used memory blocks - Dataset: heart-disease-uci")
plt.grid(True, linewidth=1, color='gainsboro', alpha=0.5)